In [1]:
import numpy as np
import pandas as pd
import numba

# set zb

In [2]:
# df = pd.read_csv('zb2.csv',index_col=0)
df = pd.read_csv('https://raw.githubusercontent.com/computational-sediment-hyd/1DSWEwithSmallerDx/main/zb2.csv',index_col=0)

dx = 0.5
Ls = np.arange(0,10000.1,dx)

Zip = np.interp(Ls, df.L, df.Z)

# condition : common

In [3]:
q = 10.0
n = 0.03
ib = 1/400
g = 9.8

h0 = (q**2*n**2/ib)**0.3 #等流水深
hc = (q**2/g)**(1/3) # 限界水深

# Nonuniform flow

In [4]:
h = np.zeros_like(Ls) #水深の配列
zb = Zip.copy()

h[0] = h0 #下流端条件
for i in range(1,len(h)):
    h[i] = h[i-1] #収束計算の初期値：一つ下流側の断面の水深
    f = 1.0 #仮値
    dfdh = 1.0 #仮値
    while np.abs(f/dfdh) > 10**(-8): # 反復計算の収束条件
        f = q**2/2.0/g/h[i]**2 + h[i] + zb[i] \
          -(q**2/2.0/g/h[i-1]**2 + h[i-1] + zb[i-1]) \
          - 0.5*(q**2*n**2/h[i]**(10/3) + q**2*n**2/h[i-1]**(10/3))*dx
        dfdh = -q**2/g/h[i]**3 + 1 + 5/3*q**2*n**2/h[i]**(13/3)*dx
        h[i] -= f/dfdh
        
hNU = h.copy()

# Unsteady flow : Colloacated

In [5]:
@numba.jit(nopython=True, parallel=False)
def UnSteadyflowCollocated(A, Q, Adown, Qup, zb, B, dx, dt, g, manning):
    imax = len(A)
    Anew, Qnew = np.zeros(imax), np.zeros(imax)
    
# continuous equation
    for i in numba.prange(1, imax-1) : Anew[i] = A[i] - dt * ( Q[i] - Q[i-1] ) / dx
        
    Anew[0], Anew[-1] = Anew[1], Adown
    
# moumentum equation
    for i in numba.prange(1,imax-1): 
        ip, ic, im = (i+1, i, i-1) 
        Cr1 = 0.5*( Q[ic]/A[ic] + Q[ip]/A[ip] )*dt/dx
        Cr2 = 0.5*( Q[ic]/A[ic] + Q[im]/A[im] )*dt/dx
        dHdx1 = ( Anew[ip]/B[ip] + zb[ip] - Anew[ic]/B[ic] - zb[ic] ) / dx
        dHdx2 = ( Anew[ic]/B[ic] + zb[ic] - Anew[im]/B[im] - zb[im] ) / dx
        dHdx = (1.0 - Cr1) * dHdx1 + Cr2 * dHdx2
        
        Qnew[ic] = Q[ic] - dt * ( Q[ic]**2/A[ic] - Q[im]**2/A[im] ) / dx  \
                   - dt * g * Anew[ic] * dHdx \
                   - dt * g * A[ic] * manning**2 * Q[ic]**2 / B[ic]**2 / ( A[ic]/B[ic] )**(10.0/3.0)
                
#     Qnew[0], Qnew[-1] = Qup, Qnew[-2]
    Qnew[0] = Qup
    
# check downstream boundary Q 
    i = -1
    ic, im = (i, i-1) 
    dHdx = ( Anew[ic]/B[ic] + zb[ic] - Anew[im]/B[im] - zb[im] ) / dx
    
    Qnew[ic] = Q[ic] - dt * ( Q[ic]**2/A[ic] - Q[im]**2/A[im] ) / dx  \
               - dt * g * Anew[ic] * dHdx \
               - dt * g * A[ic] * manning**2 * Q[ic]**2 / B[ic]**2 / ( A[ic]/B[ic] )**(10.0/3.0)
            
    return Anew, Qnew, np.abs(Anew - A).max()

In [6]:
%%time
imax = len(Ls)
dt = 0.025
totalTime = 3.0*3600.0
manning = n

# Initial & Boundary condition
zb = Zip.copy()
zb = zb[::-1]

B = np.full(imax, 1.0, dtype=float)
A = hNU[::-1]*B
Q = q*B

Qup = Q[0]
Adown = A[-1] 

tout = 600
for it in range(int(totalTime/dt)):
    A, Q, err = UnSteadyflowCollocated(A, Q, Adown, Qup, zb, B, dx, dt, g, manning)
    if (dt*it)%tout==0: print((dt*it), err)
        
hUSC = A[::-1]/B[::-1] 

0.0 0.06604200618295097
600.0 9.582725620571608e-06
1200.0 6.342622615118643e-06
1800.0 4.464433007278501e-06
2400.0 1.2445479864453546e-06
3000.0 2.220226260618574e-07
3600.0 3.2382274639530806e-08
4200.0 4.211158977085461e-09
4800.0 5.097007260701503e-10
5400.0 5.88222803798999e-11
6000.0 6.568079413682426e-12
6600.0 7.16315895488151e-13
7200.0 7.682743330406083e-14
7800.0 8.43769498715119e-15
8400.0 1.3322676295501878e-15
9000.0 1.3322676295501878e-15
9600.0 1.3322676295501878e-15
10200.0 1.3322676295501878e-15
CPU times: total: 4min 30s
Wall time: 4min 32s


# Unsteady flow : Staggered

In [7]:
@numba.jit(nopython=True, parallel=False)
def UnSteadyflowStaggered(A, Q, Adown, Qup, zb, B, dx, dt, g, manning):
    imax = len(A)
    Anew, Qnew = np.zeros(imax), np.zeros(imax)
    Qhf, Qhfnew = np.zeros(imax), np.zeros(imax) # 下流端は使わないためimax個とする。
    ie = np.zeros(imax)
    Vhf = np.zeros(imax+1)
    
# Q value cell-center to harf cell
# すべてをセルセンターで扱うため、ハーフセルへの変換では逆変換との兼ね合いで次のように定義する。
    Qhf[-1] = Q[-1]
    for i in range(imax-2, -1, -1) : 
        Qhf[i] = 2.0*Q[i] - Qhf[i+1]
    
    for i in range(imax) : 
        ie[i] = manning**2 * Q[i]**2 / B[i]**2 / ( A[i]/B[i] )**(10.0/3.0)
    
# continuous equation
    for i in range(0, imax-1) : 
        Anew[i] = A[i] - dt * ( Qhf[i+1] - Qhf[i] ) / dx
    Anew[-1] = Adown
    
    Hnew = Anew/B + zb
    
    # 逆流の場合は風上が必要
    for i in range(1, imax) : 
        Vhf[i] = Qhf[i]/A[i-1]
    Vhf[0] = Qhf[0]/A[0]
    Vhf[-1] = Q[-1]/A[-1]
    
# moumentum equation
    for i in range(1, imax): 
        dHdx = ( Hnew[i] - Hnew[i-1] ) / dx
        Ahfnew = 0.5*(Anew[i] + Anew[i-1])
        Ahf = 0.5*(A[i]+A[i-1])
        iehf = 0.5*(ie[i]+ie[i-1])
        Vp = 0.5*(Vhf[i+1] + Vhf[i])
        Vm = 0.5*(Vhf[i-1] + Vhf[i])
        
        Qhfnew[i] = Qhf[i] - dt * ( Vp*Qhf[i] - Vm*Qhf[i-1] ) / dx \
                         - dt * g * Ahfnew * dHdx \
                         - dt * g * Ahf * iehf \
            
    Qhfnew[0] = Qup
    
    for i in range(imax-1): 
        Qnew[i] = 0.5*( Qhfnew[i+1] + Qhfnew[i] )
    Qnew[-1] = Qhfnew[-1]
    
    return Anew, Qnew, np.abs(Anew - A).max()

In [8]:
%%time
imax = len(Ls)
dt = 0.025
totalTime = 3.0*3600.0
manning = n

# Initial & Boundary condition
zb = Zip.copy()
zb = zb[::-1]

B = np.full(imax, 1.0, dtype=float)
A = hNU[::-1]*B
Q = q*B

Qup = Q[0]
Adown = A[-1] 

tout = 600
for it in range(int(totalTime/dt)):
    A, Q, err = UnSteadyflowStaggered(A, Q, Adown, Qup, zb, B, dx, dt, g, manning)
    if (dt*it)%tout==0: print((dt*it), err)
        
hUSS = A[::-1]/B[::-1] 

0.0 0.0
600.0 5.136523422599737e-06
1200.0 3.427166012315297e-06
1800.0 2.422871377749658e-06
2400.0 6.408640684796296e-07
3000.0 1.0106712711177579e-07
3600.0 1.252371717441747e-08
4200.0 1.3521104236247083e-09
4800.0 1.3384138242145127e-10
5400.0 1.2504219881748213e-11
6000.0 1.1333156635373598e-12
6600.0 1.056932319443149e-13
7200.0 3.375077994860476e-14
7800.0 6.838973831690964e-14
8400.0 6.52811138479592e-14
9000.0 2.4868995751603507e-14
9600.0 2.7533531010703882e-14
10200.0 5.062616992290714e-14
CPU times: total: 4min 7s
Wall time: 4min 12s


# export

In [9]:
dfout = pd.DataFrame({'Distance':Ls, 'Zb':Zip, 'Nonuniform':hNU, 'Unsteady Collacated':hUSC,'Unsteady Staggered':hUSS})

d = dfout.to_csv('calout.csv', index_label='id')